# Brains

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from gsp_neuro import data_loading as dload 
from gsp_neuro import plotting as viz
from gsp_neuro import utils as ut
from gsp_neuro.brain import Brain

In [ ]:
data_base_directory = "/Users/hugofluhr/chuv/data/"
subject_ids = dload.get_subjects(data_base_directory)
# sub-349 has disconnected node at certain scales
subject_ids.remove('sub-CHUVL349')

scale = 1

In [ ]:
brains = [Brain(data_base_directory, sub, scale) for sub in subject_ids]

In [ ]:
for brain in brains:
    brain.load_graph()
    brain.add_signal('thickness')
    brain.add_signal('pial_lgi')

In [ ]:
consensus = Brain(data_base_directory, 'consensus', scale)
consensus.load_graph()

In [ ]:
def compute_smoothness(G, signal):
    s = signal/np.amax(signal)
    return s.T@G.L@s

# Check how smooth a signal is on self brain vs other brains

In [ ]:
signal = 'thickness'

diffs = []
for b_ref in brains:
    ref_smoothness = compute_smoothness(b_ref.G, b_ref.get_signal(signal))
    for b_comp in brains:
        if b_comp == b_ref:
            continue
        diffs.append(compute_smoothness(b_comp.G, b_ref.get_signal(signal))-ref_smoothness)

In [ ]:
plt.hist(diffs, bins=50)
plt.show()

In [ ]:
from scipy import stats 

t_statistic, p_value = stats.ttest_1samp(diffs, 0)

# Print the p-value
print("P-value:", p_value)

# Compare smoothness of Thickness vs LGI

In [ ]:
thick, lgi, diffs = [], [], []
for b in brains:
    thick_smoothness = compute_smoothness(b.G, b.get_signal('thickness'))
    lgi_smoothness = compute_smoothness(b.G, b.get_signal('pial_lgi'))
    thick.append(thick_smoothness)
    lgi.append(lgi_smoothness)
    diffs.append(thick_smoothness-lgi_smoothness)

In [ ]:
t_statistic, p_value = stats.ttest_ind(thick, lgi)

# Print the p-value
print("P-value:", p_value)

In [ ]:

data = pd.DataFrame({'Measurement': ['Cortical Thickness'] * len(thick) + ['LGI'] * len(lgi),
                     'Values': thick + lgi})

In [ ]:
sns.boxplot(x='Measurement', y='Values', data=data)

# Add labels and title
plt.xlabel('')
plt.ylabel('Smoothness on Connectome')
plt.title('Comparison of Smoothness of different signals, p-val = {:.2e}'.format(p_value))

# Show the plot
plt.savefig('../figures/boxplots_smoothness.svg')
plt.show()

## Same on rewired graph

In [ ]:
thick_ref, thick_rewired = [], []
for b in brains:
    thick_smoothness = compute_smoothness(b.G, b.get_signal('thickness'))
    rewired_b = ut.rewire(b.G)
    t_rew = compute_smoothness(rewired_b, b.get_signal('thickness'))
    thick_ref.append(thick_smoothness)
    thick_rewired.append(t_rew)

In [ ]:
data = pd.DataFrame({'Measurement': ['Brain'] * len(thick_ref) + ['Rewired Brain'] * len(thick_rewired),
                     'Values': thick_ref + thick_rewired})

In [ ]:
sns.boxplot(x='Measurement', y='Values', data=data)

# Add labels and title
plt.xlabel('')
plt.ylabel('Smoothness on Connectome')
plt.title('Comparison of Smoothness on different Graphs')

# Show the plot
plt.savefig('../figures/boxplots_smoothness.svg')
plt.show()

# Cortical Thickness

In [ ]:
len(brains)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5.5,4))
viz.plot_spectrum(brain.G,brain.get_signal('pial_lgi'),ax)
ax.set_title('Graph Fourier Transform of LGI')
ax.set_ylim([-4,4])
fig.savefig('../figures/lgi_gft.svg')


In [ ]:
viz.plot_signal_3d(brain, 'thickness', mesh_type='pial_whole', cmap="plasma", black_bg=False, title='Cortical Thickness [mm]')


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5.5,4))
viz.plot_spectrum(consensus.G,brain.get_signal('thickness'),ax)
ax.set_title('Graph Fourier Transform of Cortical Thickness')
fig.savefig('../figures/thickness_gft.svg')


In [ ]:
n_comp = 1
s = brain.get_signal('thickness')
#s_hat = consensus.G.gft(s)
s_hat = brain.G.gft(s)

In [ ]:
def lp(gft, n):
    filtered = np.zeros_like(gft)
    filtered[:n] = gft[:n]
    return filtered

In [ ]:
N = consensus.G.N

In [ ]:
s_partial = []
rec_errors = []
for n in range(N):
    lp_filtered = lp(s_hat,n)
    #s_recons = consensus.G.igft(lp_filtered)
    s_recons = brain.G.igft(lp_filtered)
    s_partial.append(s_recons)
    rec_errors.append(1-np.linalg.norm(s-s_recons)/np.linalg.norm(s))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5.5,4))
ax.plot(100*np.array(rec_errors))
ax.set_title('Recovered Cortical Thickness')
ax.set_ylabel('% of recovered signal')
ax.set_xlabel('Low-pass cutoff')
plt.show()
#fig.savefig('../figures/thick_recons.svg')

In [ ]:
from nilearn import datasets
fsaverage = datasets.fetch_surf_fsaverage(mesh="fsaverage")


In [ ]:
fsaverage.keys()

In [ ]:
from importlib import reload
reload(viz)

viz.plot_signal_2d_mini(brain, s, outpath='/Users/hugofluhr/chuv/repositories/gsp_neuro/figures/real_brain/consensus_2.svg')

In [ ]:
viz.plot_signal_3d(brain, s, mesh_type='pial_whole', cmap="plasma", black_bg=False, title='Cortical Thickness [mm]', vmin=0, vmax=3.5)

In [ ]:
n_comp = 65
viz.plot_signal_3d(brain, s_partial[n_comp], mesh_type='pial_whole', cmap="plasma", black_bg=False, title='# Components = {}'.format(n_comp), vmin=0, vmax=3.5)

In [ ]:
def eigen_reconstruction(G, signal, n):
    assert(n<=G.N)
    approx = np.zeros_like(G.L.toarray())

    for i in range(1,n):
        approx += G.e[i] * np.outer(G.U[:,i],G.U[:,i])

    error = np.linalg.norm(G.L - approx)/np.linalg.norm(G.L.toarray()) * 100
    fig, axs = plt.subplots(1,3, figsize=(15,6))
    mat = axs[0].matshow(G.e[n-1] * np.outer(G.U[:,n-1],G.U[:,n-1]))
    axs[0].set_title('Outer product of {}th eigenvector'.format(n))
    fig.colorbar(mat, ax=axs[0])

    mat = axs[1].matshow(approx)
    axs[1].set_title('Cumulative Reconstruction, error = {:.1f}%'.format(error))
    axs[1].axis(False)
    fig.colorbar(mat, ax=axs[1])
    axs[2].matshow(G.L.toarray())
    axs[2].set_title("Laplacian of the Graph")
    axs[2].axis(False)
    plt.show()

In [ ]:
lp = np.zeros((N))
lp[:20]=1.
hp = np.zeros((N))
hp[40:]=1.
heat = np.exp(-3*np.arange(N)/N)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5.5,4))
ax.plot(lp)
ax.plot(hp)
ax.plot(heat)
ax.legend(['Low-Pass','High-Pass','Heat kernel'])
ax.set_title('Filters in Frequency domain')
ax.set_ylabel('GFT coefficients')
ax.set_xlabel('Graph Frequency Index')
plt.show()
fig.savefig('../figures/filters.svg')

In [ ]:
import nibabel as nib

In [ ]:
np.unique(nib.freesurfer.read_annot("/Users/hugofluhr/chuv/data/requestionforconnectomes/lh.atlas-laus2018_desc-scale3.annot")[0])